In [45]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.append('..\..')
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, shapiro
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import  multi_conditions_plot, output_significance_stars_from_pval
from utils.plotting import get_photometry_around_event
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData, CueAlignedSidedData, ChoiceAlignedData
from set_global_params import experiment_record_path, processed_data_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def get_all_experimental_records():
    experiment_record = pd.read_csv(experiment_record_path, dtype='str')
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [ ]:
def get_trial_data(session_data):
    saving_folder = processed_data_path + session_data.mouse + '\\'
    restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
    trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
    return trial_data

In [3]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces

In [4]:
class CustomAlignedData(object):
    def __init__(self, session_data, params):
        saving_folder = processed_data_path + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.ipsi_data = ZScoredTraces(trial_data, dff, params, fiber_side_numeric, 0)
        self.ipsi_data.get_peaks()
        self.contra_data = ZScoredTraces(trial_data, dff,params, contra_fiber_side_numeric, 0)
        self.contra_data.get_peaks()

In [5]:
mouse_id = 'SNL_photo70'
date = '20220408'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

In [6]:
saving_folder = processed_data_path + mouse_id + '\\'
restructured_data_filename = mouse_id + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename)

In [7]:
cot_trials = trial_data[trial_data['Sound type'] == 0]

In [8]:
silence_trials = trial_data[trial_data['Sound type'] == 1]

In [9]:
num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

In [10]:
num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [11]:
behavioural_data0 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)

In [12]:
mouse_id = 'SNL_photo43'
date = '20210603'

all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)
fiber_options = ['left', 'right']
fiber_side = experiment_to_process['fiber_side'].values[0]
contra_choice_ind = np.where(np.array(fiber_options) != fiber_side)[0][0] + 1


In [14]:
trial_data = get_trial_data(session_data)

In [15]:
post_change_trials = trial_data[(trial_data['Trial num']>149) & (trial_data['State type'] == 1)]
silence_correct_trials =  post_change_trials[(post_change_trials['Response'] == contra_choice_ind) & (post_change_trials['Trial outcome'] == 1)] 
silence_trial_type = silence_correct_trials['Trial type'].unique()[0]
silence_trials = post_change_trials[post_change_trials['Trial type'] == silence_trial_type]
cot_trials = trial_data[(trial_data['Trial num']<=149) & (trial_data['State type'] == 1)]

In [16]:
num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

In [17]:
num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [18]:
behavioural_data1 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)

In [19]:
behavioural_data = pd.concat([behavioural_data0, behavioural_data1])

In [20]:
mouse_id = 'SNL_photo37'
date = '20210603'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

fiber_options = ['left', 'right']
fiber_side = experiment_to_process['fiber_side'].values[0]
contra_choice_ind = np.where(np.array(fiber_options) != fiber_side)[0][0] + 1


In [21]:
trial_data = get_trial_data(session_data)

post_change_trials = trial_data[(trial_data['Trial num']>149) & (trial_data['State type'] == 1)]
silence_correct_trials =  post_change_trials[(post_change_trials['Response'] == contra_choice_ind) & (post_change_trials['Trial outcome'] == 1)] 
silence_trial_type = silence_correct_trials['Trial type'].unique()[0]
silence_trials = post_change_trials[post_change_trials['Trial type'] == silence_trial_type]
cot_trials = trial_data[(trial_data['Trial num']<=149) & (trial_data['State type'] == 1)]

num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [22]:
behavioural_data1 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)
behavioural_data = pd.concat([behavioural_data, behavioural_data1])

In [23]:
behavioural_data

,mouse,missed,correct,incorrect,stimulus
0,SNL_photo70,0.000000,0.611601,0.388399,silence
1,SNL_photo70,0.000000,0.789445,0.210555,tone
0,SNL_photo43,0.012270,0.957055,0.030675,silence
1,SNL_photo43,0.000000,0.926667,0.073333,tone
0,SNL_photo37,0.011429,0.908571,0.080000,silence
1,SNL_photo37,0.000000,0.826667,0.173333,tone


In [24]:
df_for_plot_behavioural = behavioural_data.pivot(index='mouse', columns='stimulus', values='missed')

In [25]:
cols = df_for_plot_behavioural.columns.tolist()

In [26]:
cols = cols[-1:] + cols[:-1]

In [27]:
df_for_plot_behavioural = 100 - (df_for_plot_behavioural[cols] *100)

In [28]:
df_for_plot_behavioural

stimulus,tone,silence
mouse,,
SNL_photo37,100.0,98.857143
SNL_photo43,100.0,98.773006
SNL_photo70,100.0,100.000000


In [29]:
np.mean(df_for_plot_behavioural['silence'])

99.2100496640374

In [30]:
np.std(df_for_plot_behavioural['silence'])

0.5596343431027693

In [34]:
differences = df_for_plot_behavioural['tone'].values - df_for_plot_behavioural['silence'].values
shapiro(differences)

ShapiroResult(statistic=0.8011702299118042, pvalue=0.11729390174150467)

In [46]:
_, pval = ttest_rel(df_for_plot_behavioural['tone'],df_for_plot_behavioural['silence'])

In [48]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[1.5,2])

multi_conditions_plot(ax, df_for_plot_behavioural.T, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.ylabel('Missed trials (%)', fontsize=8)
#ax.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))
ax.set_xlabel(' ')

#significance stars
y = df_for_plot_behavioural.T.to_numpy().max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
stars = output_significance_stars_from_pval(pval)
ax.text(.5, y+h, stars, ha='center', fontsize=8)
ax.set_ylim([95, 100])
plt.tight_layout()